In [ ]:
# Import necessary libraries
from simphony.libraries import siepic
from simphony.die import Die
from simphony.simulation import Simulation, Laser, Detector
import matplotlib.pyplot as plt
from phidl import set_quickplot_options

In [ ]:
# Instantiate components
gc_input = siepic.GratingCoupler(name="gcinput")
y_splitter = siepic.YBranch(name="ysplit")
wg_long = siepic.Waveguide(length=150e-6, name="wglong")
wg_short = siepic.Waveguide(length=50e-6, name="wgshort")
y_recombiner = siepic.YBranch(name="yrecombiner")
gc_output = siepic.GratingCoupler(name="gcoutput")

# Instantiate Die object
die = Die(name="die1")

# Throw in the components into the Die
die.add_components([gc_input, y_splitter, wg_long, gc_output, y_recombiner, wg_short])

# Distribute the devices in the die in a grid
die.distribute_devices(direction='grid', shape=(3,2), spacing=(5,10))

# Visualize the grid arrangement
die.visualize(label_aliases=True)

In [ ]:
# We connect the components like we would usually. Simphony will take care of the routing and
# device connections for us.

# Do not show ports when plotting the layout
set_quickplot_options(False, False, label_aliases=True)

# next we connect the components to each other
y_splitter["pin1"].connect(gc_input["pin1"])

y_recombiner["pin1"].connect(gc_output["pin1"])

y_splitter["pin2"].connect(wg_long)
y_recombiner["pin3"].connect(wg_long)

y_splitter["pin3"].connect(wg_short)
y_recombiner["pin2"].connect(wg_short)

In [ ]:
# Visualize after connecting
set_quickplot_options(show_ports=False, show_subports=False, label_aliases=True)
die.visualize(show_ports=False) # Visualize after connecting

In [ ]:
# Run the layout aware monte carlo computation
with Simulation() as sim:
    l = Laser(power=1)
    l.freqsweep(187370000000000.0, 199862000000000.0)
    l.connect(gc_input['pin2'])
    d = Detector()
    d.connect(gc_output['pin2'])

    results = sim.layout_aware_simulation()

In [ ]:
# Plot the results
f = l.freqs
for run in results:
    p = []
    for sample in run:
        for data_list in sample:
            for data in data_list:
                p.append(data)
    plt.plot(f, p)

run = results[0]
p = []
for sample in run:
    for data_list in sample:
        for data in data_list:
            p.append(data)
plt.plot(f, p, 'k')
plt.title('MZI Layout Aware Monte Carlo')
plt.show()

In [ ]:
# Plot cumulative distribution function
import numpy as np

# Get maximum P
max = []
for run in results:
    p = []
    for sample in run:
        for data_list in sample:
            for data in data_list:
                p.append(data)
    max.append(10*np.log10(np.max(p)))
#sort data
x = np.sort(max)

#calculate CDF values
y = 1. * np.arange(len(max)) / (len(max) - 1)

#plot CDF
plt.plot(x, y)
plt.xlabel('x')

In [14]:
# Read circuit from SPICE file
from simphony.formatters import CircuitSiEPICFormatter
from simphony.layout import Circuit
circuit = Circuit.from_file("C:\\Users\\12269\\Documents\\GitHub\\simphony\\simphony\\plugins\\siepic\\tests\\spice\\MZI4\\MZI4_main.spi", \
    formatter=CircuitSiEPICFormatter())

In [15]:
# Add circuit to die
from simphony.die import Die
die = Die()
die.add_components(circuit._get_components())

In [18]:
for i, c in enumerate(circuit._get_components()):
    print(i, c)

0 YBranch component with pins: N$0, N$2, N$1
1 GratingCoupler component with pins: ebeam_gc_te1550_detector2, N$0
2 Waveguide component with pins: N$1, N$4
3 Waveguide component with pins: N$2, N$5
4 YBranch component with pins: N$6, N$5, N$4
5 GratingCoupler component with pins: ebeam_gc_te1550_laser1, N$3
6 Waveguide component with pins: N$6, N$3
7 SweepSimulator component with pins: to_input, to_output


In [19]:
circuit._get_components()[5].pins[0].disconnect()

In [ ]:
die.visualize()

In [21]:
# Run the layout aware monte carlo computation
with Simulation() as sim:
    l = Laser(power=1)
    l.freqsweep(187370000000000.0, 199862000000000.0)
    l.connect(circuit._get_components()[1])
    d = Detector()
    d.connect(circuit._get_components()[5])

    results = sim.layout_aware_simulation()

{<simphony.libraries.siepic.YBranch object at 0x000002237FEE6BC0>: {'x': 7.4, 'y': 127.0}, <simphony.libraries.siepic.GratingCoupler object at 0x000002237FC77A60>: {'x': 0.0, 'y': 0.0}, <simphony.libraries.siepic.YBranch object at 0x000002237FD3E710>: {'x': 89.93, 'y': 127.0}, <simphony.libraries.siepic.GratingCoupler object at 0x000002237FD3FF10>: {'x': 0.0, 'y': 0.0}}
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\12269\miniconda3\envs\test3.10\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\12269\AppData\Local\Temp\ipykernel_23224\3795129841.py", line 9, in <cell line: 2>
    results = sim.layout_aware_simulation()
  File "c:\Users\12269\Documents\GitHub\simphony\simphony\simulation.py", line 377, in layout_aware_simulation
  File "c:\Users\12269\Documents\GitHub\simphony\simphony\simulation.py", line 273, in _compute_correlated_samples
    raise ValueError(
ValueError: Incorrect number of component coordinates passed to argument "coords".

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\12269\miniconda3\envs\test3.10\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\12269\mini

In [ ]:
f = l.freqs
for run in results:
    p = []
    for sample in run:
        for data_list in sample:
            for data in data_list:
                p.append(data)
    plt.plot(f, p)

run = results[0]
p = []
for sample in run:
    for data_list in sample:
        for data in data_list:
            p.append(data)
plt.plot(f, p, 'k')
plt.title('MZI Layout Aware Monte Carlo')
plt.show()